In [10]:
import json
import random

In [2]:
with open('binvox_metadata.json', 'r') as f:
    data = json.load(f)

In [3]:
print(data.keys())

dict_keys(['cat_ids', 'model_ids'])


In [5]:
print(len(data["cat_ids"]))

17934


In [6]:
current_cat_id = ""
cat_ids = []
indexes = []
for i, cat_id in enumerate(data["cat_ids"]):
    if cat_id != current_cat_id:
        cat_ids.append(cat_id)
        indexes.append(i)
        current_cat_id = cat_id

In [13]:
print(cat_ids)
print(indexes)
cat_id_range_indexes = []
for i in range(10):
    if i != 10:
        cat_id_range_indexes.append([indexes[i], indexes[i+1]])
cat_id_range_indexes.append([indexes[10], 17934])
print(cat_id_range_indexes)
cat_id_suffled_ranges = []
for i in range(11):
    cat_range = [j for j in range(cat_id_ranges[i][0], cat_id_ranges[i][1])]
    random.shuffle(cat_range)
    cat_id_suffled_ranges.append(cat_range)

['02818832', '02876657', '02880940', '03001627', '03046257', '03325088', '03593526', '03636649', '03642806', '03797390', '04379243']
[0, 204, 668, 809, 6726, 7260, 7774, 8328, 9542, 9973, 10182]
[[0, 204], [204, 668], [668, 809], [809, 6726], [6726, 7260], [7260, 7774], [7774, 8328], [8328, 9542], [9542, 9973], [9973, 10182], [10182, 17934]]


In [15]:
print(cat_id_suffled_ranges[0][0:20])
train_indexes = []
test_indexes = []
for cat_range in cat_id_suffled_ranges:
    train_number = int(len(cat_range) * 0.9)
    train_indexes.extend(cat_range[:train_number])
    test_indexes.extend(cat_range[train_number:])
print(len(train_indexes), len(test_indexes), len(train_indexes)+len(test_indexes))

[117, 91, 181, 203, 139, 124, 165, 3, 47, 148, 183, 40, 97, 135, 164, 172, 64, 200, 104, 98]
16134 1800 17934


In [16]:
with open('train_indexes.json', 'w') as f:
    json.dump(train_indexes, f)
with open('test_indexes.json', 'w') as f:
    json.dump(test_indexes, f)

In [21]:
with open('train_indexes.json', 'r') as f:
    indexes = json.load(f)
    print(indexes[0:20])

[117, 91, 181, 203, 139, 124, 165, 3, 47, 148, 183, 40, 97, 135, 164, 172, 64, 200, 104, 98]
